In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import torch

In [7]:
# importing the final df of all parties.
df_final = pd.read_csv('../data/processed/df_final_v1.csv')

In [10]:
df_nlp = df_final[['user.name', 'full_text']]

In [24]:
df_nlp.head(3)

,user.name,full_text
0,Partido Popular,“Se acaba la sesión y no sabemos si el Gobiern...
1,Partido Popular,La gestión de la ministra de Trabajo: 6 millon...
2,Partido Popular,“Teniendo en cuenta que 1 de cada 4 españoles ...


In [ ]:
# first try without cleaning anything

In [11]:
model = tf.keras.Sequential()

In [14]:
model.add(hub.KerasLayer("https://tfhub.dev/google/nnlm-es-dim50/2",
                         input_shape=[], 
                         dtype=tf.string,
                         trainable=False))

In [16]:
model.add(tf.keras.layers.Dense(16, activation='relu'))

In [17]:
model.add(tf.keras.layers.Dense(1))

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48832000  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,832,833
Trainable params: 833
Non-trainable params: 48,832,000
_________________________________________________________________


In [19]:
from sklearn.model_selection import train_test_split

In [21]:
tweets_train, tweets_test = train_test_split(df_nlp)

In [23]:
tweets_train.shape, tweets_test.shape

((3507, 2), (1169, 2))

In [ ]:
# entrenamos el modelo
model.fit(x=tweets_train['user.name'])

In [ ]:
# BERT: 5 stars

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
